In [1]:
import pandas as pd
import numpy as np
import datetime
from tqdm.notebook import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import requests
from bs4 import BeautifulSoup
import time
import re
from urllib.request import urlopen
from itertools import combinations, permutations
import matplotlib.pyplot as plt
import pickle

In [2]:
pd.set_option('display.max_columns', 50)

In [3]:
# 自作モジュールインポート
import module.Preprocessing as preprocessing
import module.Scraping as scraping
import module.Simulation as simulation

In [4]:
# ラベルエンコーダーのインポート
le_horse = pickle.load(open('labelencoder/le_horse.pickle', 'rb'))
le_jockey = pickle.load(open('labelencoder/le_jockey.pickle', 'rb'))

In [5]:
# モデルのインポート
lgb_clf_danger = pickle.load(open('model/dangerous_pred.pkl', 'rb'))

In [6]:
m_race_results = pd.read_pickle("data/preprocessing_data/m_race_results.pickle")
p_race_results = pd.read_pickle("data/preprocessing_data/p_race_results.pickle")
p_horse_results = pd.read_pickle("data/preprocessing_data/p_horse_results.pickle")

## 出馬表のデータをスクレイピングして予想する

### 注意事項

- 元データにrace_idが存在しない場合は実行できないため、全てのレースを予想出来るわけではない（新馬は無理）
- 特徴量の「人気」はbeatifulsoopで取得できなかったので手打ちで入力する


In [7]:
# ここは手打ちで入力する
# https://race.netkeiba.com/race/shutuba.html?race_id=202307011211&rf=race_list

shutuba_id = "202305010411"
date = "2023/02/05"

# ↑のリンクから人気を順番に手打ち
popular =  [14,4,1,3,5,7,13,9,11,12,16,15,10,8,2,6]

In [8]:
# 新馬が含まれていないか判定
# horse_idが表示された場合は予想出来ないもしくは、除外して予想する
horse_id_list = scraping.shutuba_horse_id_scraping(shutuba_id)
results = set(horse_id_list) ^ set(p_horse_results[p_horse_results.index.isin(horse_id_list)].index.unique())
if results == set():
  print("新馬なし")
else:
  results

新馬なし


出馬表のスクレイピングと前処理

In [9]:
shutuba = scraping.shutuba_scraping([shutuba_id], date)
shutuba["人気"] = popular
p_shutuba = preprocessing.shutuba_preprocessing(shutuba)


  0%|          | 0/1 [00:00<?, ?it/s]

出馬表に出頭馬の過去レース結果をマージする

In [10]:
m_shutuba = preprocessing.merge_race_with_horse(p_shutuba, p_horse_results)

カテゴリ変数化

In [11]:
d_shutuba = preprocessing.dummy_with_category(le_horse, le_jockey, m_shutuba, m_race_results)

In [12]:
# 日付をドロップ
d_shutuba = d_shutuba.drop(columns="date", axis=1)

In [13]:
proba_danger = simulation.predict_proba(lgb_clf_danger, d_shutuba)

[LightGBM] [Warning] Unknown parameter: axis


In [14]:
df_score = shutuba
df_score["score"] = proba_danger
df_score[["馬番", "馬名", "人気", "score"]].sort_values("人気")

,馬番,馬名,人気,score
202305010411,3,ジャスティンカフェ,1,2.964991
202305010411,15,ナミュール,2,1.029304
202305010411,4,ピンハイ,3,0.843056
202305010411,2,ウインカーネリアン,4,0.415178
202305010411,5,インダストリア,5,0.370246
202305010411,16,プレサージュリフト,6,0.302373
202305010411,6,マテンロウオリオン,7,-0.794345
202305010411,14,エアロロノア,8,0.216809
202305010411,8,ファルコニア,9,-0.546786
202305010411,13,サクラトゥジュール,10,-0.360403
